<a href="https://colab.research.google.com/github/Thomas4Daniel/medical-insurance-cost-analysis/blob/main/Basic_of_DFT_calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My name is **Dr. Ashfaq Ahmad**, working in the field of structure biology and bioinformatics. The DFT notebook is designed for teaching and research purposes. If you want to use this pipeline for commercial purpose, please contact. Detailed video tutorial of this notebook can be found on [Bioinformatic Insights](https://youtu.be/qGqXS4aWgAg).

**Announcement**

If you need assistance in developing a pipeline for your research protocol in a paid capacity, feel free to reach out from the channel page or github. I’d be happy to help!

In [ ]:
#@title Install necessary libraries
!pip install pyscf ase matplotlib numpy ipywidgets pubchempy rdkit reportlab

# Density Functional Theory (DFT) Analysis

Density Functional Theory (DFT) is a computational quantum mechanical modeling method used to investigate the electronic structure of many-body systems, especially atoms, molecules, and the condensed phases. DFT is among the most popular and versatile methods available in condensed-matter physics, computational physics, and computational chemistry.

## Basic Concepts of DFT

1. **Hohenberg-Kohn Theorems**:
   - The ground state properties of a many-electron system are uniquely determined by the electron density.
   - The electron density that minimizes the total energy is the true ground state electron density.

2. **Kohn-Sham Equations**:
   - The Kohn-Sham equations are a set of self-consistent equations that allow us to find the electron density of a system.

3. **Exchange-Correlation Functionals**:
   - The exchange-correlation functional accounts for the non-classical part of the electron-electron interaction.

In [ ]:
#@title Now, let's set up a simple DFT calculation using _PySCF_
from pyscf import gto, dft

# Define a molecule (e.g., water)
mol = gto.Mole()
mol.atom = '''
O 0 0 0
H 0 1 0
H 0 0 1
'''
mol.basis = 'sto-3g'
mol.build()

# Perform DFT calculation
mf = dft.RKS(mol)
mf.xc = 'b3lyp'  # Use B3LYP functional
mf.kernel()

# Print the total energy
print("Total Energy (B3LYP):", mf.e_tot)

In [ ]:
#@title Visualize the molecular orbitals using _PySCF_ and _matplotlib_
import matplotlib.pyplot as plt
import numpy as np

# Plot the HOMO (Highest Occupied Molecular Orbital)
homo_index = mol.nelectron // 2 - 1  # Index of the HOMO
homo = mf.mo_coeff[:, homo_index]

# Generate a grid for visualization
from pyscf import tools
tools.cubegen.orbital(mol, 'homo.cube', homo)

# Load and plot the cube file
from ase.io.cube import read_cube_data
data, atoms = read_cube_data('homo.cube')

# Plot the HOMO
plt.imshow(data[:, :, data.shape[2]//2], cmap='RdBu')
plt.colorbar()
plt.title('HOMO of Water Molecule')
plt.show()

In [ ]:
#@title Calculate for the molecule of your choice
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, dft
from pyscf.tools import cubegen
import matplotlib.pyplot as plt
from ase.io.cube import read_cube_data
import ipywidgets as widgets
from IPython.display import display

# Create a text box and button for user input
cid_input = widgets.Text(value='', placeholder='Enter PubChem CID (e.g., 2519 for caffeine)', description='CID:')
calculate_button = widgets.Button(description="Calculate HOMO")

# Output area to display results
output = widgets.Output()

# Function to handle button click
def on_calculate_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        try:
            cid = int(cid_input.value)
            print(f"Fetching molecule with CID: {cid}")

            # Fetch the molecule and generate 3D coordinates
            atoms, coords = fetch_and_optimize_molecule(cid)

            # Perform DFT calculation and visualize the HOMO
            calculate_and_visualize_homo(atoms, coords)
        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the function to the button
calculate_button.on_click(on_calculate_button_click)

# Display the input box, button, and output area
display(cid_input, calculate_button, output)

# Function to fetch molecule from PubChem and generate 3D coordinates
def fetch_and_optimize_molecule(cid):
    # Fetch the molecule from PubChem
    compound = pcp.Compound.from_cid(cid)
    smiles = compound.canonical_smiles
    print(f"Molecule Name: {compound.iupac_name}")
    print(f"Molecular Formula: {compound.molecular_formula}")
    print(f"SMILES: {smiles}")

    # Convert SMILES to 3D structure using RDKit
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize geometry

    # Extract atomic coordinates
    conf = mol.GetConformer()
    atoms = [mol.GetAtomWithIdx(i).GetSymbol() for i in range(mol.GetNumAtoms())]
    coords = [conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())]

    return atoms, coords

# Function to perform DFT calculation and visualize HOMO
def calculate_and_visualize_homo(atoms, coords):
    # Define the molecule in PySCF
    mol_pyscf = gto.Mole()
    mol_pyscf.atom = [[atom, (coord.x, coord.y, coord.z)] for atom, coord in zip(atoms, coords)]
    mol_pyscf.basis = 'sto-3g'  # Basis set
    mol_pyscf.build()

    # Perform DFT calculation
    mf = dft.RKS(mol_pyscf)
    mf.xc = 'b3lyp'  # Use B3LYP functional
    mf.kernel()

    # Print the total energy
    print(f"Total Energy (B3LYP): {mf.e_tot}")

    # Generate cube file for the HOMO
    homo_index = mol_pyscf.nelectron // 2 - 1  # Index of the HOMO
    cubegen.orbital(mol_pyscf, 'homo.cube', mf.mo_coeff[:, homo_index])

    # Load and visualize the HOMO
    homo_data, homo_atoms = read_cube_data('homo.cube')
    plt.figure(figsize=(6, 6))
    plt.imshow(homo_data[:, :, homo_data.shape[2] // 2], cmap='RdBu', origin='lower')
    plt.colorbar(label='Electron Density')
    plt.title('HOMO of the Molecule')
    plt.show()

**Warning**: Below code generates a kind of prelimary resut report. However, this code is in testing phase, and I personally did not test it on different compounds, therefore, I strongly suggest to please consult an expert, in case you want to publish it.

In [ ]:
#@title Test Compound of your choice from the PubChem with **Preliminary Report**
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, dft
from pyscf.tools import cubegen
import matplotlib.pyplot as plt
from ase.io.cube import read_cube_data
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from mpl_toolkits.mplot3d import Axes3D

# Create a text box and button for user input
cid_input = widgets.Text(value='', placeholder='Enter PubChem CID (e.g., 2519 for caffeine)', description='CID:')
calculate_button = widgets.Button(description="Generate HOMO Report")

# Output area to display results
output = widgets.Output()

# Function to handle button click
def on_calculate_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        try:
            cid = int(cid_input.value)
            print(f"Fetching molecule with CID: {cid}")

            # Fetch the molecule and generate 3D coordinates
            atoms, coords = fetch_and_optimize_molecule(cid)

            # Perform DFT calculation and analyze the HOMO
            report, homo_data, homo_atoms = calculate_and_analyze_homo(atoms, coords)

            # Display the report
            print("\n--- HOMO Analysis Report ---")
            print(report)

            # Plot the HOMO
            plot_homo(homo_data, homo_atoms)

            # Save the report as a PDF
            save_report_as_pdf(report, homo_data, homo_atoms)
            print("\nReport saved as 'homo_report.pdf'.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the function to the button
calculate_button.on_click(on_calculate_button_click)

# Display the input box, button, and output area
display(cid_input, calculate_button, output)

# Function to fetch molecule from PubChem and generate 3D coordinates
def fetch_and_optimize_molecule(cid):
    # Fetch the molecule from PubChem
    compound = pcp.Compound.from_cid(cid)
    smiles = compound.canonical_smiles
    print(f"Molecule Name: {compound.iupac_name}")
    print(f"Molecular Formula: {compound.molecular_formula}")
    print(f"SMILES: {smiles}")

    # Convert SMILES to 3D structure using RDKit
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize geometry

    # Extract atomic coordinates
    conf = mol.GetConformer()
    atoms = [mol.GetAtomWithIdx(i).GetSymbol() for i in range(mol.GetNumAtoms())]
    coords = [conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())]

    return atoms, coords

# Function to perform DFT calculation and analyze HOMO
def calculate_and_analyze_homo(atoms, coords):
    # Define the molecule in PySCF
    mol_pyscf = gto.Mole()
    mol_pyscf.atom = [[atom, (coord.x, coord.y, coord.z)] for atom, coord in zip(atoms, coords)]
    mol_pyscf.basis = 'sto-3g'  # Basis set
    mol_pyscf.build()

    # Perform DFT calculation
    mf = dft.RKS(mol_pyscf)
    mf.xc = 'b3lyp'  # Use B3LYP functional
    mf.kernel()

    # Generate cube file for the HOMO
    homo_index = mol_pyscf.nelectron // 2 - 1  # Index of the HOMO
    cubegen.orbital(mol_pyscf, 'homo.cube', mf.mo_coeff[:, homo_index])

    # Load and analyze the HOMO cube file
    homo_data, homo_atoms = read_cube_data('homo.cube')

    # Analyze the HOMO data
    report = analyze_homo_data(homo_data, homo_atoms)

    # Add total energy to the report
    report += f"\nTotal Energy (B3LYP): {mf.e_tot:.6f} Hartree"

    return report, homo_data, homo_atoms

# Function to analyze HOMO data and generate a report
def analyze_homo_data(homo_data, homo_atoms):
    # Calculate the total electron density
    total_density = np.sum(homo_data)

    # Find the maximum and minimum electron density values
    max_density = np.max(homo_data)
    min_density = np.min(homo_data)

    # Analyze symmetry and localization
    # For simplicity, we assume the molecule is symmetric if the density is evenly distributed
    symmetry = "symmetric" if np.allclose(homo_data, np.flip(homo_data)) else "asymmetric"

    # Check if the HOMO is localized or delocalized
    # For simplicity, we assume delocalization if the density is spread across multiple atoms
    localization = "delocalized" if np.std(homo_data) < 0.5 else "localized"

    # Generate the report
    report = f"""
--- HOMO Analysis Report ---
1. Electron Density Distribution:
   - Total Electron Density: {total_density:.6f}
   - Maximum Electron Density: {max_density:.6f}
   - Minimum Electron Density: {min_density:.6f}

2. Symmetry:
   - The HOMO is {symmetry}.

3. Localization:
   - The HOMO is {localization}.

4. Reactivity Insights:
   - Regions with high electron density are potential sites for nucleophilic attack.
   - The HOMO represents the most loosely bound electrons, which are likely to participate in chemical reactions.
"""

    return report

# Function to plot the HOMO
def plot_homo(homo_data, homo_atoms):
    # Plot a 2D slice of the HOMO
    plt.figure(figsize=(8, 6))
    plt.imshow(homo_data[:, :, homo_data.shape[2] // 2], cmap='RdBu', origin='lower')
    plt.colorbar(label='Electron Density')
    plt.title('HOMO of the Molecule (2D Slice)')
    plt.savefig('homo_2d_slice.png')  # Save the 2D plot
    plt.show()

    # Plot a 3D isosurface of the HOMO with red-to-blue colormap
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Create a grid for the 3D plot
    x, y, z = np.mgrid[:homo_data.shape[0], :homo_data.shape[1], :homo_data.shape[2]]

    # Normalize the electron density for better color mapping
    norm = plt.Normalize(vmin=homo_data.min(), vmax=homo_data.max())

    # Use a red-to-blue colormap
    scatter = ax.scatter(x, y, z, c=homo_data.flatten(), cmap='RdBu', norm=norm, alpha=0.3)  # Adjust transparency
    fig.colorbar(scatter, label='Electron Density')
    ax.set_title('HOMO of the Molecule (3D Isosurface)')
    plt.savefig('homo_3d_isosurface.png')  # Save the 3D plot
    plt.show()

# Function to save the report as a PDF
def save_report_as_pdf(report, homo_data, homo_atoms):
    # Create a PDF file
    pdf = canvas.Canvas("homo_report.pdf", pagesize=letter)
    pdf.setFont("Helvetica", 12)

    # Add the report text
    pdf.drawString(50, 750, "HOMO Analysis Report")
    y = 730
    for line in report.split('\n'):
        pdf.drawString(50, y, line)
        y -= 15

    # Add the 2D plot
    pdf.drawString(50, y - 20, "2D Slice of the HOMO:")
    pdf.drawImage('homo_2d_slice.png', 50, y - 220, width=400, height=200)

    # Add the 3D plot
    pdf.drawString(50, y - 240, "3D Isosurface of the HOMO:")
    pdf.drawImage('homo_3d_isosurface.png', 50, y - 440, width=400, height=200)

    # Save the PDF
    pdf.save()

## Analysis and Discussion of Results

Let's discuss these results, such as the total energy, molecular orbitals, and electron density

- **Total Energy**: The total energy of the system is calculated using the B3LYP functional.
- **Molecular Orbitals**: The HOMO (Highest Occupied Molecular Orbital) is visualized, showing the electron density distribution.
- **Electron Density**: The electron density can be analyzed to understand the chemical bonding and reactivity of the molecule.

# Interpreting the HOMO (Highest Occupied Molecular Orbital) Figure

The **HOMO (Highest Occupied Molecular Orbital)** figure visualizes the electron density distribution in the highest energy orbital that is occupied by electrons in a molecule. This orbital is significant because it plays a key role in chemical reactions, particularly in processes like **electron donation** (e.g., in redox reactions) and **bond formation**.

---

## What Does the HOMO Represent?
- The HOMO represents the **most loosely bound electrons** in the molecule.
- These electrons are the most likely to participate in chemical reactions, such as forming bonds with other molecules or donating electrons.
- The shape and distribution of the HOMO provide insights into the **reactivity** and **electronic properties** of the molecule.

---

## Key Features to Look For in the HOMO Figure

### 1. **Electron Density Distribution**
- The **bright regions** (often colored red or blue in the plot) represent areas of high electron density.
- The **dim regions** (often white or transparent) represent areas of low or no electron density.
- The electron density is typically concentrated around **atoms** or **bonds** where the electrons are most likely to be found.

### 2. **Symmetry and Shape**
- The HOMO often reflects the **symmetry** of the molecule. For example, in a symmetric molecule like benzene, the HOMO will have a symmetric distribution of electron density.
- The shape of the HOMO can be **bonding**, **antibonding**, or **non-bonding**:
  - **Bonding orbitals**: Electron density is concentrated between atoms, indicating stabilizing interactions.
  - **Antibonding orbitals**: Electron density is concentrated outside the bond region, indicating destabilizing interactions.
  - **Non-bonding orbitals**: Electron density is localized on a single atom (e.g., lone pairs).

### 3. **Localization vs. Delocalization**
- If the electron density is **localized** on a specific atom or bond, it suggests that the HOMO is associated with a particular functional group or region of the molecule.
- If the electron density is **delocalized** over multiple atoms or bonds, it indicates that the electrons are shared across a larger region of the molecule (e.g., in conjugated systems like benzene or polyenes).

---

## Interpreting the HOMO for Chemical Reactivity
- **Nucleophilic Sites**: Regions with high electron density in the HOMO are potential sites for **nucleophilic attack** (donating electrons).
- **Electron Donation**: The HOMO is the orbital from which electrons are most easily donated in a reaction.
- **Reactivity Trends**: Molecules with **higher energy HOMOs** are generally more reactive because their electrons are less tightly bound.

---

## Example: HOMO of Water (H₂O)
Let’s take the example of water (H₂O), which we used in the DFT calculation earlier.

### **HOMO Characteristics for Water**:
- The HOMO of water is primarily localized on the **oxygen atom**, representing the **lone pairs** of electrons.
- The electron density is concentrated around the oxygen, with little to no density on the hydrogen atoms.
- This indicates that the oxygen atom is the most likely site for **electron donation** (e.g., in hydrogen bonding or nucleophilic reactions).

### **Visualization**:
- In the HOMO plot, you would see bright regions (high electron density) around the oxygen atom and dim regions around the hydrogens.

---

## Example: HOMO of Benzene (C₆H₆)
For a more complex molecule like benzene:

### **HOMO Characteristics for Benzene**:
- The HOMO of benzene is **delocalized** over the entire ring, reflecting the **π-electron system**.
- The electron density is evenly distributed above and below the plane of the ring.
- This delocalization indicates that benzene is highly stable and less reactive compared to molecules with localized HOMOs.

### **Visualization**:
- In the HOMO plot, you would see a symmetric, donut-shaped electron density distribution above and below the ring.

---

## How to Use the HOMO in Analysis
- **Predict Reactivity**: Identify regions of high electron density to predict where the molecule is likely to react.
- **Compare Molecules**: Compare HOMOs of different molecules to understand their relative reactivity or stability.
- **Understand Bonding**: Use the HOMO to analyze bonding patterns, such as conjugation or lone pairs.

---

## Limitations of the HOMO
- The HOMO only provides information about the **highest occupied orbital**. For a complete picture of reactivity, you should also consider the **LUMO (Lowest Unoccupied Molecular Orbital)**, which represents the lowest energy orbital that can accept electrons.
- The HOMO does not account for **dynamic effects** (e.g., solvent interactions or temperature), which can also influence reactivity.

---

## Summary
- The HOMO figure shows the distribution of the most loosely bound electrons in a molecule.
- High electron density regions indicate potential sites for reactivity (e.g., nucleophilic attack).
- The shape and symmetry of the HOMO reflect the molecule’s electronic structure and bonding.
- By analyzing the HOMO, you can gain insights into the molecule’s chemical behavior and reactivity.

## Conclusion

In this tutorial, we have introduced the basic concepts of Density Functional Theory (DFT) and demonstrated how to perform DFT calculations on a simple molecule using `PySCF`. We also visualized the molecular orbitals and analyzed the results.

## DFT Calculations for Vibrational Frequencies and Thermodynamics Properties

### Further Reading
- [PySCF Documentation](https://pyscf.org/)
- [Density Functional Theory: A Practical Introduction by David Sholl and Janice A. Steckel](https://onlinelibrary.wiley.com/doi/book/10.1002/9780470447710)